<a href="https://colab.research.google.com/github/vimuth97/FYP-Brain-Tumor-Classification/blob/main/Basic%20pathology%20model/rejoin_patches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import h5py
!apt update && apt install -y openslide-tools
!pip install openslide-python 
import openslide 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#Get all image names in the Data directory
def getAllImageNames(root_dir):
    files = os.listdir(root_dir)
    fileNames = []
    for file in files:
        file = file.split(".")
        fileNames.append(file[0])
    return fileNames

# Get all patchs which are sub region of given image name
def getAllPatchFilesForImage(name,patches_dir):
    files = os.listdir(patches_dir)
    patchSet = []
    for file in files:
        fileName = file.split(".")
        fileName = fileName[0].split("-")
        if fileName[0] == name:
            patchSet.append(file)
    return patchSet

def getWSIDimentions(path):
    wsi = openslide.OpenSlide(path)
    wsi_h, wsi_w = wsi.level_dimensions[0]
    return [wsi_h,wsi_w]

#getAllImageNames('gdrive/My Drive/Data_Directory')
#print(getAllPatchFilesForImage('CPM19_CBICA_AAB_1','gdrive/My Drive/RESULTS_DIRECTORY/patches'))

In [ ]:
def rejoinPatches(save_dir,image_dir,patches_dir):
    imageFiles = getAllImageNames(image_dir)
    for name in imageFiles:
        patchFiles = getAllPatchFilesForImage(name,patches_dir)
        print(name,"\nNo of Patches: "+str(len(patchFiles)))
        allCoordinates = np.empty((0,2), int)
        img_dim = getWSIDimentions(os.path.join(image_dir, name+".tiff"))
        
        for patchFile in patchFiles:
            patch_file_path = os.path.join(patches_dir, patchFile)
            #Extract coordinates of the regions
            patchFileName = patchFile.split(".")
            patchFileName = patchFileName[0].split("-")
            coordinates = [int(x) for x in patchFileName[1].split("_")] # h,w

            # Open patch file
            with h5py.File(patch_file_path,'r') as hdf5_file:
                data_set = hdf5_file['coords']
                for cd in data_set:
                    allCoordinates = np.append(allCoordinates,[[cd[0]+coordinates[0] , cd[1]+coordinates[1]]], axis = 0) # x+w , y+h
                print(np.shape(data_set),np.shape(allCoordinates))
            
        print(np.shape(allCoordinates))
        complete_patch_path = os.path.join(save_dir, name)+'.h5'
        patch_file = h5py.File(complete_patch_path, "a")

        dtype = allCoordinates.dtype
        img_shape = allCoordinates.shape
        maxshape = (None, 2)
        dset = patch_file.create_dataset('coords', 
                                shape=img_shape, maxshape=maxshape,  chunks=(1, 2), dtype=dtype)
        dset[:] = allCoordinates
        dset.attrs['wsi_name'] = name
        dset.attrs['downsample'] = [1.,1.]
        dset.attrs['downsampled_level_dim'] = img_dim
        dset.attrs['level_dim'] = img_dim
        dset.attrs['patch_level'] = 0
        dset.attrs['patch_size'] = 256
        dset.attrs['save_path'] = save_dir

        patch_file.close()



        print()


In [ ]:
save_dir = 'gdrive/My Drive/RESULTS_DIRECTORY/rejoined_patches'
image_dir = 'gdrive/My Drive/Data_Directory'
patches_dir = 'gdrive/My Drive/RESULTS_DIRECTORY/patches'

rejoinPatches(save_dir,image_dir,patches_dir)

In [ ]:
files = os.listdir('gdrive/My Drive/RESULTS_DIRECTORY/patches') 
allCoordinates = np.empty((0,2), int)
for f in files:
    file_path = os.path.join('gdrive/My Drive/RESULTS_DIRECTORY/patches', f)   
    fileName = f.split(".")
    fileName = fileName[0].split("-")
    coordinates = [int(x) for x in fileName[1].split("_")]
    print(coordinates)
    with h5py.File(file_path,'r') as hdf5_file:
        dset = hdf5_file['coords']
        for name, value in dset.attrs.items():
            print(name, value,type(value))

In [ ]:
files = os.listdir('gdrive/My Drive/RESULTS_DIRECTORY/rejoined_patches') 
#allCoordinates = np.empty((0,2), int)
for f in files:
    file_path = os.path.join('gdrive/My Drive/RESULTS_DIRECTORY/rejoined_patches', f)   
    fileName = f.split(".")
    fileName = fileName[0].split("-")
    #coordinates = [int(x) for x in fileName[1].split("_")]
    #print(coordinates)
    with h5py.File(file_path,'r') as hdf5_file:
        dset = hdf5_file['coords']
        for name, value in dset.attrs.items():
            print(name, value,type(value))
    print()

In [ ]:
file_path = 'gdrive/My Drive/RESULTS_DIRECTORY/patches/CPM19_CBICA_ALZ_1-52000_44000.h5' 
#fileName = f.split(".")
#fileName = fileName[0].split("-")
#coordinates = [int(x) for x in fileName[1].split("_")]
#print(coordinates)
with h5py.File(file_path,'r') as hdf5_file:
    dset = hdf5_file['coords']
    for name, value in dset.attrs.items():
        print(name, value,type(value))
    print(dset[:])